# [CSC8101] Big Data Analytics - 2022 Spark Coursework

By-PARTH SAGAR
210431117

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
trips.take(10)

Trips dataset loaded!
 ---
 Size: XXL
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-XXL-delta/
 Number of trips (dataset rows): 132,396,785
 
Out[2]: [Row(index=0, VendorID=1.0, tpep_pickup_datetime='2019-01-01 00:46:40', tpep_dropoff_datetime='2019-01-01 00:53:20', passenger_count=1.0, trip_distance=1.5, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=151, DOLocationID=239, payment_type=1.0, fare_amount=7.0, extra=0.5, mta_tax=0.5, tip_amount=1.65, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=9.95, congestion_surcharge=None, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None),
 Row(index=1, VendorID=1.0, tpep_pickup_datetime='2019-01-01 00:59:47', tpep_dropoff_datetime='2019-01-01 01:18:59', passenger_count=1.0, trip_distance=2.6, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=239, DOLocationID=246, payment_type=1.0, fare_amount=14.0, extra=0.5, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=16.3, congestion_surcharge=None, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None),
 Row(index=2, VendorID=2.0, tpep_pickup_datetime='2018-12-21 13:48:30', tpep_dropoff_datetime='2018-12-21 13:52:40', passenger_count=3.0, trip_distance=0.0, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=236, DOLocationID=236, payment_type=1.0, fare_amount=4.5, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=5.8, congestion_surcharge=None, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None),
 Row(index=3, VendorID=2.0, tpep_pickup_datetime='2018-11-28 15:52:25', tpep_dropoff_datetime='2018-11-28 15:55:45', passenger_count=5.0, trip_distance=0.0, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=193, DOLocationID=193, payment_type=2.0, fare_amount=3.5, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=7.55, congestion_surcharge=None, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None),
 Row(index=4, VendorID=2.0, tpep_pickup_datetime='2018-11-28 15:56:57', tpep_dropoff_datetime='2018-11-28 15:58:33', passenger_count=5.0, trip_distance=0.0, RatecodeID=2.0, store_and_fwd_flag='N', PULocationID=193, DOLocationID=193, payment_type=2.0, fare_amount=52.0, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=55.55, congestion_surcharge=None, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None),
 Row(index=5, VendorID=2.0, tpep_pickup_datetime='2018-11-28 16:25:49', tpep_dropoff_datetime='2018-11-28 16:28:26', passenger_count=5.0, trip_distance=0.0, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=193, DOLocationID=193, payment_type=2.0, fare_amount=3.5, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=5.76, improvement_surcharge=0.3, total_amount=13.31, congestion_surcharge=None, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None),
 Row(index=6, VendorID=2.0, tpep_pickup_datetime='2018-11-28 16:29:37', tpep_dropoff_datetime='2018-11-28 16:33:43', passenger_count=5.0, trip_distance=0.0, RatecodeID=2.0, store_and_fwd_flag='N', PULocationID=193, DOLocationID=193, payment_type=2.0, fare_amount=52.0, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=55.55, congestion_surcharge=None, cab_type='Y', lpep_pickup_datetime=None, lpep_dropoff_datetime=None, ehail_fee=None, trip_type=None),
 Row(index=7, VendorID=1.0, tpep_pickup_datetime='2019-01-01 00:21:28', tpep_dropoff_datetime='2019-01-01 00:28:37', passenger_count=1.0, trip_distance=1.3, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=163, DOLocationID=229, payment_type=1.0, fare_amount=6.5, extra=0.5, mta_tax=0.5, tip_amount=1.25, tolls_amount=0

In [0]:
print_count(trips)

Row count: 132,396,785

In [0]:
# dataset schemas
trips.printSchema()

root
-- index: long (nullable = true)
-- VendorID: double (nullable = true)
-- tpep_pickup_datetime: string (nullable = true)
-- tpep_dropoff_datetime: string (nullable = true)
-- passenger_count: double (nullable = true)
-- trip_distance: double (nullable = true)
-- RatecodeID: double (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- PULocationID: long (nullable = true)
-- DOLocationID: long (nullable = true)
-- payment_type: double (nullable = true)
-- fare_amount: double (nullable = true)
-- extra: double (nullable = true)
-- mta_tax: double (nullable = true)
-- tip_amount: double (nullable = true)
-- tolls_amount: double (nullable = true)
-- improvement_surcharge: double (nullable = true)
-- total_amount: double (nullable = true)
-- congestion_surcharge: double (nullable = true)
-- cab_type: string (nullable = true)
-- lpep_pickup_datetime: string (nullable = true)
-- lpep_dropoff_datetime: string (nullable = true)
-- ehail_fee: double (nullable = true)
-- trip_type: double (nullable = true)

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
151,239,1.5,1.0,9.95
239,246,2.6,1.0,16.3
236,236,0.0,3.0,5.8
193,193,0.0,5.0,7.55
193,193,0.0,5.0,55.55


In [0]:
zone_names.printSchema()

root
-- LocationID: long (nullable = true)
-- Borough: string (nullable = true)
-- Zone: string (nullable = true)
-- service_zone: string (nullable = true)

In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.438 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
import pyspark

Removing zeros by using filter function

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    #filtering the values. removing the null values.
    df=df.filter((df['trip_distance']!=0) | ((df['passenger_count']!=0) & (df['total_amount']!=0)))
    #As total amount should never be null 
    df=df.filter((df['total_amount']!=0))
    return df


In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)
# printing the count of the trips after removing zeros 
print_count(trips_11)

Row count: 132,260,957

In [0]:
from pyspark.sql.functions import abs

In [0]:
from pyspark.sql.functions import lit
import statistics

In [0]:
# creating a column with name trip distance 1 for adding null values 
trips_11=trips_11.withColumn("trip_distance1", lit(None))

In [0]:
print_count(trips_11)

Row count: 132,260,957

Removing outliers by calculating Z value

In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    # calculating the median
    median=df.agg(pf.percentile_approx(df['trip_distance'], 0.5)).alias("median").collect()[0][0]
    # subtaracting median from the column
    sub=df['trip_distance']-median
    # adding the value of column in the trip_distance1 
    df=df.withColumn("trip_distance1", sub)
    # taking the absolute values of the column
    df.trip_distance1=abs(df['trip_distance1'])
    # calculating median to calculate mad
    madmedian=df.agg(pf.percentile_approx(df['trip_distance1'], 0.5)).alias("median").collect()[0][0]
    # calculating mad value
    mad=(1.438*madmedian)
    # calculating trip_distance column
    df.trip_distance1=df.trip_distance1/mad
    # calculating absolute values
    df.trip_distance1=abs(df.trip_distance1)
    # filtering the values which are greater than 3.5
    df=df.filter(df['trip_distance1']<3.5)
    
    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)
# removing the outliers and printing the count
print_count(trips_12)
# trips_12.take(10)

Row count: 113,565,161

## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    # left join on the basis of pickup location id
    df=df.join(zones_df,df.PULocationID== zones_df.LocationID, "left")
    # renaming the column
    df=df.withColumnRenamed("LocationID","start")
    # renaming the column
    df=df.withColumnRenamed("Zone","startzone")
    # left join on the basisi of drop location
    df=df.join(zones_df,df.DOLocationID== zones_df.LocationID, "left")
    #renaming the column
    df=df.withColumnRenamed("LocationID","end")
    #renaming the column
    df=df.withColumnRenamed("Zone","endzone")
    return df

In [0]:
# execute task 2.1
#calling the join function
trips_21 = t21_join_zones(trips_11, zones_df = zone_names)
#printing the count of trips 
print_count(trips_21)

Row count: 132,260,957

In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    # dividing the total amount by trip_distance to calculate unit profitability
    div=df.total_amount/df.trip_distance
    # assigning the value of unit profitability
    df=df.withColumn("unit_profitability", div)
    return df

In [0]:
# execute task 2.2
#calculating profit
trips_22 = t22_calc_profit(trips_21)
# printing the rows count
print_count(trips_22)

Row count: 132,260,957

## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

summarizing the trips and calculating values

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    # selecting the main coulumns to save time
    df=df.select(df.startzone, df.PULocationID, df.DOLocationID, df.endzone, df.unit_profitability,df.passenger_count)
    # grouping on the basis of pickup and drop off zones and calculating profitability, trips and passenger count
    gdf = df.groupBy(df.PULocationID,df.DOLocationID)
    df1=gdf.agg( pf.count(df.startzone),pf.mean(df.unit_profitability), pf.sum(df.passenger_count))

    return df1

In [0]:
# execute task 3.1
# calling summarise function
graph = t31_summarise_trips(trips_22)
# printing the count of graph
print_count(graph)

Row count: 59,627

In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    # group by  pickup location id and aggregation of the total trips, mean  profit and total passenger count
    gdf = df.groupBy(df.PULocationID)
    df1=gdf.agg( pf.sum(df['count(startzone)']),pf.mean(df['avg(unit_profitability)']), pf.sum(df['sum(passenger_count)']))
    # left join on the basis location id
    df=df1.join(zones_df,df1.PULocationID==zones_df.LocationID, 'left')
    # renaming the columns
    df=df.withColumnRenamed("sum(count(startzone))","trips_count")
    df=df.withColumnRenamed("avg(avg(unit_profitability))","average_unit_profit")
    df=df.withColumnRenamed("sum(sum(passenger_count))","total_passengers")
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    #descending the trips count and returning the top 10 values 
    df_zones=df_zones.orderBy(pf.col('trips_count').desc())
    trips_count=df_zones.take(10)
    return trips_count

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    #descending the average unit profit and returning the top 10 values 
    df_zones=df_zones.orderBy(pf.col('average_unit_profit').desc())
    average_unit_profit=df_zones.take(10)
    return average_unit_profit

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    #descending the total passenger count and returning the top 10 values 
    df_zones=df_zones.orderBy(pf.col('total_passengers').desc())
    total_passengers=df_zones.take(10)
    return total_passengers

In [0]:
# execute task 3.2
# calling the functions
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
#displaying the values of trips
display(top10_trips)

PULocationID,trips_count,average_unit_profit,total_passengers,LocationID,Borough,Zone,service_zone
237,5565367,5.731121393786021,8288668.0,237,Manhattan,Upper East Side South,Yellow Zone
236,5133997,5.270868417978195,7648525.0,236,Manhattan,Upper East Side North,Yellow Zone
161,4896136,6.835602103320631,7485661.0,161,Manhattan,Midtown Center,Yellow Zone
186,4386664,6.260195245882188,6550668.0,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone
162,4326027,6.4597764452960575,6498112.0,162,Manhattan,Midtown East,Yellow Zone
230,3867855,7.2415726975302075,6134643.0,230,Manhattan,Times Sq/Theatre District,Yellow Zone
170,3835380,6.788539994841746,5719171.0,170,Manhattan,Murray Hill,Yellow Zone
48,3804024,6.761326038377758,5848454.0,48,Manhattan,Clinton East,Yellow Zone
132,3773317,7.254275789211485,5919007.0,132,Queens,JFK Airport,Airports
142,3753491,5.719840828478446,5714842.0,142,Manhattan,Lincoln Square East,Yellow Zone


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
#displaying the values of profit

display(top10_profit)

PULocationID,trips_count,average_unit_profit,total_passengers,LocationID,Borough,Zone,service_zone
1,11346,75.19307598426424,17521.0,1,EWR,Newark Airport,EWR
5,764,38.11952040113714,742.0,5,Staten Island,Arden Heights,Boro Zone
96,1537,17.838349008347294,1396.0,96,Queens,Forest Park/Highland Park,Boro Zone
84,245,16.72528532418096,183.0,84,Staten Island,Eltingville/Annadale/Prince's Bay,Boro Zone
265,0,15.930682515596063,96931.0,265,Unknown,null,null
204,149,15.450854439425235,131.0,204,Staten Island,Rossville/Woodrow,Boro Zone
172,200,14.894519854525143,212.0,172,Staten Island,New Dorp/Midland Beach,Boro Zone
264,1062695,14.691130191132736,1560032.0,264,Unknown,NV,null
120,2071,13.8753064535616,2304.0,120,Manhattan,Highbridge Park,Boro Zone
191,20670,13.750465817456696,14561.0,191,Queens,Queens Village,Boro Zone


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
#displaying the values of passenger

display(top10_passenger)

PULocationID,trips_count,average_unit_profit,total_passengers,LocationID,Borough,Zone,service_zone
237,5565367,5.731121393786011,8288668.0,237,Manhattan,Upper East Side South,Yellow Zone
236,5133997,5.270868417978192,7648525.0,236,Manhattan,Upper East Side North,Yellow Zone
161,4896136,6.835602103320628,7485661.0,161,Manhattan,Midtown Center,Yellow Zone
186,4386664,6.260195245882183,6550668.0,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone
162,4326027,6.45977644529606,6498112.0,162,Manhattan,Midtown East,Yellow Zone
230,3867855,7.241572697530205,6134643.0,230,Manhattan,Times Sq/Theatre District,Yellow Zone
132,3773317,7.2542757892114835,5919007.0,132,Queens,JFK Airport,Airports
48,3804024,6.761326038377756,5848454.0,48,Manhattan,Clinton East,Yellow Zone
170,3835380,6.788539994841747,5719171.0,170,Manhattan,Murray Hill,Yellow Zone
142,3753491,5.719840828478446,5714842.0,142,Manhattan,Lincoln Square East,Yellow Zone


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'L'
DATA_FORMAT = 'delta'
WITH_TASK_12 = False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

Trips dataset loaded!
 ---
 Size: L
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-L-delta/
 Number of trips (dataset rows): 41,953,716

In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[31]: [[Row(PULocationID=237, trips_count=1846251, average_unit_profit=60.62277085684773, total_passengers=2609882.0, LocationID=237, Borough='Manhattan', Zone='Upper East Side South', service_zone='Yellow Zone'),
 Row(PULocationID=236, trips_count=1741036, average_unit_profit=15.16535913992203, total_passengers=2465982.0, LocationID=236, Borough='Manhattan', Zone='Upper East Side North', service_zone='Yellow Zone'),
 Row(PULocationID=161, trips_count=1345821, average_unit_profit=38.330865796422145, total_passengers=1930725.0, LocationID=161, Borough='Manhattan', Zone='Midtown Center', service_zone='Yellow Zone'),
 Row(PULocationID=186, trips_count=1257008, average_unit_profit=28.473139364443075, total_passengers=1771032.0, LocationID=186, Borough='Manhattan', Zone='Penn Station/Madison Sq West', service_zone='Yellow Zone'),
 Row(PULocationID=162, trips_count=1187234, average_unit_profit=46.08166715708264, total_passengers=1681068.0, LocationID=162, Borough='Manhattan', Zone='Midtown East', service_zone='Yellow Zone'),
 Row(PULocationID=142, trips_count=1154349, average_unit_profit=84.88091020042395, total_passengers=1660519.0, LocationID=142, Borough='Manhattan', Zone='Lincoln Square East', service_zone='Yellow Zone'),
 Row(PULocationID=170, trips_count=1140014, average_unit_profit=75.87296200387618, total_passengers=1606192.0, LocationID=170, Borough='Manhattan', Zone='Murray Hill', service_zone='Yellow Zone'),
 Row(PULocationID=239, trips_count=1108998, average_unit_profit=85.19293480970109, total_passengers=1599589.0, LocationID=239, Borough='Manhattan', Zone='Upper West Side South', service_zone='Yellow Zone'),
 Row(PULocationID=141, trips_count=1090450, average_unit_profit=30.35513976920627, total_passengers=1541141.0, LocationID=141, Borough='Manhattan', Zone='Lenox Hill West', service_zone='Yellow Zone'),
 Row(PULocationID=48, trips_count=1027295, average_unit_profit=86.26414056670421, total_passengers=1485379.0, LocationID=48, Borough='Manhattan', Zone='Clinton East', service_zone='Yellow Zone')],
 [Row(PULocationID=1, trips_count=2668, average_unit_profit=1286.2114974000065, total_passengers=4117.0, LocationID=1, Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(PULocationID=132, trips_count=103792, average_unit_profit=506.1543655639854, total_passengers=149716.0, LocationID=132, Borough='Queens', Zone='JFK Airport', service_zone='Airports'),
 Row(PULocationID=201, trips_count=418, average_unit_profit=271.7545888138123, total_passengers=279.0, LocationID=201, Borough='Queens', Zone='Rockaway Park', service_zone='Boro Zone'),
 Row(PULocationID=219, trips_count=2055, average_unit_profit=230.0330605167803, total_passengers=1213.0, LocationID=219, Borough='Queens', Zone='Springfield Gardens South', service_zone='Boro Zone'),
 Row(PULocationID=14, trips_count=7658, average_unit_profit=202.89378267448237, total_passengers=3470.0, LocationID=14, Borough='Brooklyn', Zone='Bay Ridge', service_zone='Boro Zone'),
 Row(PULocationID=71, trips_count=11997, average_unit_profit=196.83034091531584, total_passengers=2551.0, LocationID=71, Borough='Brooklyn', Zone='East Flatbush/Farragut', service_zone='Boro Zone'),
 Row(PULocationID=55, trips_count=6567, average_unit_profit=194.7209638109694, total_passengers=3647.0, LocationID=55, Borough='Brooklyn', Zone='Coney Island', service_zone='Boro Zone'),
 Row(PULocationID=204, trips_count=14, average_unit_profit=192.78282760472607, total_passengers=12.0, LocationID=204, Borough='Staten Island', Zone='Rossville/Woodrow', service_zone='Boro Zone'),
 Row(PULocationID=10, trips_count=6824, average_unit_profit=192.7085429967722, total_passengers=2750.0, LocationID=10, Borough='Queens', Zone='Baisley Park', service_zone='Boro Zone'),
 Row(PULocationID=135, trips_count=3701, average_unit_profit=190.05149168031517, total_passengers=1956.0, LocationID=135, Borough='Queens', Zone='Kew Gardens Hills', service_zone='Boro Zone')],
 [Row(PULocationID=237, trips_count=1846251, average_uni

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.9 | 15.5 | 41.9 | 90.5 | 132.4 |
| execution time  (w/o 1.2)  | 9.69 | 108.6 | 1044.6 | 1457.4 | 1634.4 |
| execution time              | 11.81 | 169.8 | 1525.2 | 2092.2 | 2457.6 |
| sec / 1M records (w/o 1.2)  | 3.3413  | 7.006  | 24.9  | 16.099  | 12.34  |
| sec / 1M records            | 4.072  | 10.95  | 36.4 | 23.11  | 18.56  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2.9 | 15.5 | 41.9 | 90.5 | 132.4 |
 execution time   (w/o 1.2)  | 7.76 | 9.99 | 19.72 | 40.42 | 55.78 |
| execution time              | 7.90 | 13.19 | 24.6 | 47.73 | 65.4 |
| sec / 1M records (w/o 1.2)  | 2.675 | 0.6445 | 0.4706 | 0.446  | 0.421  |
| sec / 1M records            | 2.72  | 0.85  | 0.587  | 0.527  | 0.4939  |